In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math

%matplotlib inline

In [3]:
data = pd.read_csv("../data/iris_data.csv", header=None)

In [4]:
data

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


Splitting the dataset into X and Y variables

In [8]:
X_train, y_train = data[[0, 1, 2, 3]], data[4]

In [9]:
mu = X_train.mean(axis=0)
sigma  = X_train.std(axis=0)

X_train = (X_train - mu) / sigma

In [10]:
y_train = y_train.replace({
  "Iris-setosa": 0,
  "Iris-versicolor": 1,
  "Iris-virginica": 2
})

/var/folders/rk/_r6t3b5j0jd1cjytp0py3tv00000gn/T/ipykernel_14951/4187090884.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = y_train.replace({


One-hot encoding the Y labels

In [11]:
Y_onehot = pd.get_dummies(y_train)

In [12]:
X_train

,0,1,2,3
0,-0.897674,1.028611,-1.336794,-1.308593
1,-1.139200,-0.124540,-1.336794,-1.308593
2,-1.380727,0.336720,-1.393470,-1.308593
3,-1.501490,0.106090,-1.280118,-1.308593
4,-1.018437,1.259242,-1.336794,-1.308593
...,...,...,...,...
145,1.034539,-0.124540,0.816888,1.443121
146,0.551486,-1.277692,0.703536,0.918985
147,0.793012,-0.124540,0.816888,1.050019
148,0.430722,0.797981,0.930239,1.443121


In [15]:
num_features = X_train.shape[1]
num_classes = 3

W = np.zeros((num_features, num_classes))
b = np.zeros((1, num_classes))

In [ ]:
def compute_logits(X, W, b):
  return np.dot(X, W) + b

(1, 3)

In [26]:
def softmax(z):
  exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
  return exp_z / np.sum(exp_z, axis=1, keepdims=True)

### Cross-Entropy Loss

For multi-class classification with one-hot encoded labels, the cross-entropy loss is defined as:

$$
\mathcal{L} = -\frac{1}{m} \sum_{i=1}^{m} \sum_{j=1}^{C} y_{ij} \cdot \log(\hat{y}_{ij})
$$

Where:
- \( m \) is the number of samples  
- \( C \) is the number of classes  
- \( y_{ij} \) is the true one-hot encoded label (1 if sample \( i \) belongs to class \( j \), otherwise 0)  
- \( \hat{y}_{ij} \) is the predicted probability for sample \( i \), class \( j \)


In [31]:
def cross_entropy_loss(Y_pred, Y_true):
  m = Y_true.shape[0]
  return -np.sum(Y_true * np.log(Y_pred * 1e-15)) / m